# Unsupervised Clustering Methods for Meteorological European Configurations/ Patterns


<span style="color: yellow;"> - prendo solo un sub set dei dati per la velocità (fatto in -> 1.1), poi sarà da prendre tutto il dataset (30.07.25)  </span>  
<span style="color: yellow;">- vedere se togliere i percentili in 1.1</span>  
<span style="color: yellow;">- In 2.2 vedere se standard scaler va bene o se è emglio usare robust scaler, per ora dovrebbe andare bene</span>  
<span style="color: yellow;">- eventualmente nella standardizzazione posso demarcare la cella sopra che ha le funzioni per il calcolo della memoria</span> 

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from statsmodels.stats.outliers_influence import variance_inflation_factor


## 1 Caricamento Dati e Analisi Iniziale

In [2]:
try:
    ds = xr.open_dataset('era5_2000_2004.grib', engine= 'cfgrib') # XArray DataSet
    print("Dataset loaded successfully.")
except Exception as e:
    print(f"Error loading dataset: {e}")

Dataset loaded successfully.


In [3]:
print("Overview of the dataset:")
print(f"   • Variabili: {list(ds.data_vars.keys())}")
print(f"   • Coordinate: {list(ds.coords.keys())}")

Overview of the dataset:
   • Variabili: ['z', 't', 'u', 'v']
   • Coordinate: ['number', 'time', 'step', 'isobaricInhPa', 'latitude', 'longitude', 'valid_time']


In [4]:
# Dimenision details
print("Dimension details:")
if 'latitude' in ds.dims:
    print(f"   • Latitude: {ds.dims['latitude']} points ({ds.latitude.min().values:.1f}° - {ds.latitude.max().values:.1f}°)")
if 'longitude' in ds.dims:
    print(f"   • Longitude: {ds.dims['longitude']} points ({ds.longitude.min().values:.1f}° - {ds.longitude.max().values:.1f}°)")
if 'time' in ds.dims:
    print(f"   • Time: {ds.dims['time']} steps ({pd.to_datetime(ds.time.values[0]).strftime('%Y-%m-%d')} - {pd.to_datetime(ds.time.values[-1]).strftime('%Y-%m-%d')})")
if 'isobaricInhPa' in ds.dims:
    print(f"   • Pressure levels: {ds.dims['isobaricInhPa']} levels ({list(ds.isobaricInhPa.values)} hPa)")

#Variables 
print("Variables in the dataset:")
for var in ds.data_vars:
    var_data = ds[var]
    print(f"   • {var}: {var_data.dims} - {var_data.attrs.get('long_name', 'N/A')}")
    print(f"     └─ Units: {var_data.attrs.get('units', 'N/A')}")


Dimension details:
   • Latitude: 201 points (20.0° - 70.0°)
   • Longitude: 321 points (-40.0° - 40.0°)
   • Time: 1827 steps (2000-01-01 - 2004-12-31)
   • Pressure levels: 3 levels ([np.float64(850.0), np.float64(500.0), np.float64(250.0)] hPa)
Variables in the dataset:
   • z: ('time', 'isobaricInhPa', 'latitude', 'longitude') - Geopotential
     └─ Units: m**2 s**-2
   • t: ('time', 'isobaricInhPa', 'latitude', 'longitude') - Temperature
     └─ Units: K
   • u: ('time', 'isobaricInhPa', 'latitude', 'longitude') - U component of wind
     └─ Units: m s**-1
   • v: ('time', 'isobaricInhPa', 'latitude', 'longitude') - V component of wind
     └─ Units: m s**-1


In [5]:
# Total dimensionality
total_spatial_points = 1
for dim in ['latitude', 'longitude']:
    if dim in ds.dims:
        total_spatial_points *= ds.dims[dim]

total_features = len(ds.data_vars) * ds.dims.get('isobaricInhPa', 1) * total_spatial_points
print("DIMENSIONALITY:")
print(f"   • Spatial points: {total_spatial_points}")
print(f"   • Total features per timestep: {total_features:,}")
print(f"   • Temporal samples: {ds.dims.get('time', 1)}")

DIMENSIONALITY:
   • Spatial points: 64521
   • Total features per timestep: 774,252
   • Temporal samples: 1827


Spatial points: punti griglia nello spazio. 
La regione osservata è suddivisa in una griglia regolare (0.25° x 0.25°), per ogni punto nella grigli avengono misurate le variabili

Total features per timestep: numero di variabili (features) in totale in ogni istante di tempo

Temporal samples: punti temporali nel dataset (365 giorni per 5 anni)

Posso trasformarlo in una matrice per il clustering:  
shape = (temporal_samples, total_features_per_timestep)
       = (1827, 774252)  
Ogni riga = una mappa meteorologica


### 1.1 Check quality in data

In [6]:
# Missing values check: already verified to be 0
print("MISSING VALUES: No missing values (previously verified)")

MISSING VALUES: No missing values (previously verified)


There are no missing values in the dataset

In [7]:
variables = list(ds.data_vars.keys())
print(f"STATISTICS:\nVariables found: {variables}\n")

for var in variables:
    print(f"\n {var.upper()}:")
    
    var_min = float(ds[var].min().values)
    var_max = float(ds[var].max().values)
    var_mean = float(ds[var].mean().values)
    var_std = float(ds[var].std().values)

    print(f"      • Min: {var_min:.3f}")
    print(f"      • Max: {var_max:.3f}")
    print(f"      • Mean: {var_mean:.3f}")
    print(f"      • Std: {var_std:.3f}")

STATISTICS:
Variables found: ['z', 't', 'u', 'v']


 Z:
      • Min: 7495.887
      • Max: 108808.312
      • Mean: 57405.691
      • Std: 36090.105

 T:
      • Min: 7495.887
      • Max: 108808.312
      • Mean: 57405.691
      • Std: 36090.105

 T:
      • Min: 198.893
      • Max: 308.547
      • Mean: 252.359
      • Std: 24.976

 U:
      • Min: 198.893
      • Max: 308.547
      • Mean: 252.359
      • Std: 24.976

 U:
      • Min: -63.877
      • Max: 112.808
      • Mean: 8.984
      • Std: 13.904

 V:
      • Min: -63.877
      • Max: 112.808
      • Mean: 8.984
      • Std: 13.904

 V:
      • Min: -91.157
      • Max: 89.342
      • Mean: -0.238
      • Std: 11.834
      • Min: -91.157
      • Max: 89.342
      • Mean: -0.238
      • Std: 11.834


## 2 Preprocessing and Feature Selection

### 2.1 Preparing Data Matrix

Ho 4 variabili: z, u, v, t
Per ogni variabile: var[time, pressure, latitude, longitude]

Ogni variabile ha 64521 valori per timestamp (3 × 201 × 321 = 64521 punti spaziali)(3 lv. di pressione x 201 lat x 321 lon)  
Con 4 variabili ⇒ ogni riga della matrice finale avrà:

    4 × 64521 = 774252 colonne (features)

Un timestamp è come un pacco 3D: (3, 201, 321, 4)
Un pacco per ogni giorno dal 01/01/2000 al 31/12/2004 → 1827 pacchi

#### Struttura inziale dei dati

**Scatola** = Dataset   
dentro la scatola di sono dei blocchi di fogli

**Un blocco di fogli** = un signolo giorno ( da 1 gennaio 2000 a 21 dic 2004)  -> 1827 giorni  
il blocco di fogli è formato da 4 fogli uno per ogni variabile

**Un foglio contiene i valori di una variabile** =  variabili: u, v, z, t -> 4 variabili  

Ogni foglio contiene i valori di quella variabile presi in ogni singolo punto dello 'spazio' definito dalla longitudine e dalla laitudine. Quindi in ogni foglio c'è il valore di quella variabile in ognuno dei 201(lat) × 321(lon). Una specie di tabella.  -> 64521 punti spaziali

**Solo che questa tabella di valori è presa per ognuno dei 3 livelli di pressione** = 850 hPa, 500 hPa, 250 hPa -> 3 lv di pressione

**TOT= 774252 valori per blocco**   x 1827 giorni

Per ogni variabile:  
__per ogni livello di pressione:  
____per ogni lat:  
______per ogni lon:  
________prendi il valore  

Immagina il foglio come una tabella con 774252 colonne, e solo 1 riga, che rappresenta tutte le misure spaziali per quel giorno.
Se metti insieme tutti i 1827 fogli, ottieni una matrice finale di forma (1827, 774252). Ogni riga è un giorno. Ogni colonna è una variabile a una certa posizione e pressione.


#### Struttura finale dei dati

L’obiettivo è trasformare tutto in una tabella 2D:

           feature_1  feature_2  ...  feature_774252  
time_1 →      ...        ...             ...  
time_2 →      ...        ...             ...  
  ⋮                             
time_1827 →   ...        ...             ...  

Righe: 1827 (una per ogni timestep)  
Colonne: 774,252 (una per ogni combinazione di punto spaziale × variabile)  

Ogni colonna è, ad esempio:  
"z_850hPa_lat37.5_lon12.0"  
"u_500hPa_lat42.5_lon18.0"  

In [8]:
# Dataset convertito in array 2D 
def prepare_data_matrix(dataset):
    """Converte xarray dataset in matrice 2D con float32"""
    data_matrices = {}
    
    for var in dataset.data_vars:
        print(f"   • Processando {var}...")
        var_data = dataset[var]
        
        if 'time' in var_data.dims:
            spatial_dims = [dim for dim in var_data.dims if dim != 'time']
            if spatial_dims:                                             # From: var[time=1827, pressure=3, lat=201, lon=321]
                stacked = var_data.stack(features=spatial_dims)          # To:  var[time=1827, features=193563] 
                matrix = stacked.values.astype(np.float32)  # ← float32 dimezza memoria
            else:
                matrix = var_data.values.reshape(-1, 1).astype(np.float32)
        else:
            matrix = var_data.values.flatten().reshape(1, -1).astype(np.float32)
        
        data_matrices[var] = matrix
    
    # Concatena in float32
    all_matrices = list(data_matrices.values())
    combined_matrix = np.concatenate(all_matrices, axis=1)
    
    return combined_matrix, data_matrices

print("PREPARING DATA MATRIX")
X, data_matrices = prepare_data_matrix(ds)
print(f"Data Matrix: {X.shape} (samples, features)")

PREPARING DATA MATRIX
   • Processando z...
   • Processando t...
   • Processando t...
   • Processando u...
   • Processando u...
   • Processando v...
   • Processando v...
Data Matrix: (1827, 774252) (samples, features)
Data Matrix: (1827, 774252) (samples, features)


Prima (per la variabile z):  
z[time=0, pressure=850, lat=37.5, lon=12.0] = 1234.5  
z[time=0, pressure=500, lat=37.5, lon=12.0] = 5678.9  
z[time=0, pressure=250, lat=37.5, lon=12.0] = 9876.1  
...  

Dopo lo stack:  
z[time=0, feature_0] = 1234.5  # (850hPa, lat37.5, lon12.0)  
z[time=0, feature_1] = 5678.9  # (500hPa, lat37.5, lon12.0)    
z[time=0, feature_2] = 9876.1  # (250hPa, lat37.5, lon12.0)  
...  

Concatenazione finale:

X[time=0] = [z_features (tti i valori di z)... | t_features... | u_features... | v_features...]


### 2.2 Standardization

Per ora uso StandardScaler poichè i valori sono ben distribuiti. Ma nel caso la PCA o il Kmeans venissero strai posso provare ad utilizzare RobustScaler che è più robusto agli outliers.

Se la memoria è un problema, potresti valutare l’uso di uno scaler "incrementale" (sklearn.preprocessing.StandardScaler supporta partial_fit per i batch) — oppure ridurre la dimensionalità prima con PCA.  

Se ho problemi di RAM, posso:

    Salvare i batch su disco dopo la standardizzazione (es. con npy o HDF5).

    Usare joblib o dask per gestire dati più grandi della RAM.

    Ridurre le feature prima della standardizzazione (es. PCA incrementale).

# PULIZIA MEMORIA E MONITORAGGIO
import gc
import psutil
import os

def get_memory_usage():
    """Ottiene l'uso della memoria corrente"""
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    return memory_info.rss / 1024 / 1024 / 1024  # GB

def clean_memory():
    """Pulisce la memoria e forza garbage collection"""
    gc.collect()
    print(f"   • Memoria dopo pulizia: {get_memory_usage():.2f} GB")

print("PULIZIA MEMORIA PRE-STANDARDIZZAZIONE")
print(f"   • Memoria iniziale: {get_memory_usage():.2f} GB")
print(f"   • Dimensione matrice X: {X.nbytes / 1024 / 1024 / 1024:.2f} GB")

# Pulizia forzata
clean_memory()

# Verifica che X sia in float32
if X.dtype != np.float32:
    print("   • AVVISO: Convertendo X in float32...")
    X = X.astype(np.float32)
    clean_memory()

print("   • Memoria pronta per standardizzazione")

In [ ]:
print("STANDARDIZATION - Ultra Memory-Safe (Batch size 1)")
import gc

# BATCH SIZE = 1 per evitare qualsiasi crash
batch_size = 1
print(f"   • Batch size: {batch_size} ")
#print(f"   • Memoria pre-fit: {get_memory_usage():.2f} GB")

scaler = StandardScaler()

# Step 1: Calcolo statistiche un campione alla volta
print("   • Fase 1: Calcolo statistiche (1 campione alla volta)...")
for i in range(0, X.shape[0], batch_size):
    # Evita ANY copia - usa slice diretto
    scaler.partial_fit(X[i:i+1])
    
    # Log ogni 100 campioni per evitare spam
    if i % 100 == 0:
        progress = (i / X.shape[0]) * 100
        #print(f"     → Progresso fit: {progress:.1f}% ({i}/{X.shape[0]}) - Memoria: {get_memory_usage():.2f} GB")
        gc.collect()

print("   • partial_fit completed")
#clean_memory()

# Step 2: Trasformazione diretta in-place (ZERO copie temporanee)
print("   • Fase 2: Trasformazione in-place (ZERO copie)...")
for i in range(0, X.shape[0]):
    # Trasformazione diretta su singola riga SENZA variabili temporanee
    X[i:i+1] = scaler.transform(X[i:i+1])
    
    # Log ogni 100 campioni
    if i % 100 == 0:
        progress = (i / X.shape[0]) * 100
        #print(f"     → Progresso transform: {progress:.1f}% ({i}/{X.shape[0]}) - Memoria: {get_memory_usage():.2f} GB")

        # Garbage collection ogni 200 campioni
        if i % 200 == 0:
            gc.collect()

print(f"   • Standardization completed!")
#clean_memory()

print(f"   • Dataset shape: {X.shape}")
#print(f"   • Memoria finale: {get_memory_usage():.2f} GB")

# Verifica su sample ultra-piccolo
print("   • Verifica standardizzazione (mini-sample):")
# Solo 5 campioni casuali per evitare sovraccarico
test_indices = [0, 100, 500, 1000, 1500]
for idx in test_indices:
    if idx < X.shape[0]:
        sample_mean = X[idx].mean()
        print(f"     → Campione {idx}: mean = {sample_mean:.6f}")

# Test prime 3 features su primi 3 campioni
print("   • Test prime 3 features (primi 3 campioni):")
mini_sample = X[:3, :3]
print(f"   • Mini-sample means: {mini_sample.mean(axis=0)}")
print(f"   • Mini-sample stds: {mini_sample.std(axis=0)}")

STANDARDIZATION - Ultra Memory-Safe (Batch size 1)
   • Batch size: 1 (estrema sicurezza)
   • Fase 1: Calcolo statistiche (1 campione alla volta)...
   • partial_fit completed
   • Fase 2: Trasformazione in-place (ZERO copie)...
   • Standardization completed!
   • Dataset shape: (1827, 774252)
   • Verifica standardizzazione (mini-sample):
     → Campione 0: mean = -0.308231
     → Campione 100: mean = -0.152127
     → Campione 500: mean = -0.068365
     → Campione 1000: mean = 0.178275
     → Campione 1500: mean = -0.324477
   • Test prime 3 features (primi 3 campioni):
   • Mini-sample means: [-2.7600496 -2.7559578 -2.7526739]
   • Mini-sample stds: [0.5020489  0.49900958 0.49649367]


: 

Stato della Standardizzazione:  
Globalmente corretta: La maggior parte dei campioni ha media ≈ 0  
Alcune variazioni: Normali per dati meteorologici reali  
Deviazione standard ≈ 0.5: Ragionevole per dati standardizzati  